# **Predictive Default Risk Assessor V.01**

# TODO

* Base model 
* Comparison
* Specialised
* For small entities - Examples?
* Backtest - All sectors 
* Understanding the model across all sectors/industries
* Any markets - consumer goods, industries
* UI last step after backtesting

In [1]:
import pandas as pd
import numpy as np

In [2]:
model_inputs = {
    "profitability": {
        "class_weight": 0.30,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (40, float("inf")),
                    (35, 39),
                    (30, 34),
                    (25, 29),
                    (20, 24),
                    (15, 19),
                    (10, 14),
                    (5, 9),
                    (float("-inf"), 0)
                ],
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.55,
        "weights": [0.4, 0.3, 0.3],
        "metrics": {
            "tot_debt_to_tot_eqy": {
                "lower_is_better": True,
                "thresholds": [
                     (float("-inf"), 2.0),
                     (2.0, 16.0),
                     (16.0, 24.0),
                     (24.0, 33.0),
                     (33.0, 43.0),
                     (43.0, 54.0),
                     (54.0, 68.0),
                     (68.0, 94.0),
                     (94.0, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (float("-inf"), 0.09),
                    (0.09, 0.49),
                    (0.49, 0.9),
                    (0.9, 1.36),
                    (1.36, 1.68),
                    (1.68, 2.26),
                    (2.26, 3.27),
                    (3.27, 4.4),
                    (4.4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
    "efficiency": {
        "class_weight": 0.15,
        "weights": [0.5, 0.5],
        "metrics": {
            "return_on_asset": {
                "lower_is_better": False,
                "thresholds": [
                    (0.15, float("inf")),
                    (0.10, 0.15),
                    (0.08, 0.10),
                    (0.06, 0.08),
                    (0.04, 0.06),
                    (0.02, 0.04),
                    (0.00, 0.02),
                    (-0.02, 0.00),
                    (float("-inf"), -0.02)
                ],
            },
            "asset_turnover": {
                "lower_is_better": False,
                "thresholds": [
                    (4.0, float("inf")),
                    (3.0, 4.0),
                    (2.0, 3.0),
                    (1.5, 2.0),
                    (1.0, 1.5),
                    (0.75, 1.0),
                    (0.5, 0.75),
                    (0.25, 0.5),
                    (float("-inf"), 0.25)
                ],
            },
        },
    },
}

In [3]:
model_metrics = [
    'oper_margin', 'tot_debt_to_tot_eqy', 'tot_debt_to_ebitda',
    'ebitda_to_tot_int_exp', 'return_on_asset', 'asset_turnover',
]

In [4]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        credit_ratings = {
            "Aaa": 2.5,
            "Aa": 3.5,
            "A": 4.5,
            "Baa": 5.5,
            "Ba": 6.5,
            "B": 7.5,
            "Caa": 8.5,
            "Ca": 9.5,
            "C": float("inf")
        }
        return next(rating for rating, threshold in credit_ratings.items() if weighted_score <= threshold)

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [80]:
def determine_credit_rating(weighted_score):
    credit_ratings = [
        (2.5, "Aaa"),
        (3.5, "Aa"),
        (4.5, "A"),
        (5.5, "Baa"),
        (6.5, "Ba"),
        (7.5, "B"),
        (8.5, "Caa"),
        (9.5, "Ca"),
        (10, "C"),
    ]

    for threshold, rating in credit_ratings:
        if weighted_score < threshold:
            return rating

In [97]:
features = pd.read_excel("dataset/features.xlsx", index_col=0)
targets = pd.read_excel("dataset/target.xlsx", index_col=0)
features.columns = features.columns.str.lower()

In [98]:
features

,asset_turnover,bs_lt_borrow,bs_st_borrow,bs_total_liabilities,bs_tot_asset,cfo_to_tot_debt,ebitda,ebitda_to_interest_expn,ebitda_to_revenue,ebitda_to_tot_int_exp,...,return_on_cap,return_on_inv_capital,sales_growth,sales_rev_turn,short_and_long_term_debt,total_equity,tot_debt_to_ebitda,tot_debt_to_tot_asset,tot_debt_to_tot_cap,tot_debt_to_tot_eqy
AGL SJ Equity,0.4577,2.770832e+05,31777.2720,6.378648e+05,1.215280e+06,0.3649,121616.1358,4.7608,21.4962,3.4697,...,3.8440,3.3318,-12.7171,5.657577e+05,3.088605e+05,5.774149e+05,2.5667,25.4148,34.8493,53.4902
ANG SJ Equity,0.5640,3.889976e+04,5113.5840,8.099552e+04,1.492984e+05,0.4531,18864.5579,9.6807,25.5943,9.6807,...,-12.4486,-2.1459,1.7996,8.457203e+04,4.401335e+04,6.830287e+04,1.8845,29.4801,39.1870,64.4385
ANH SJ Equity,0.2747,1.354406e+06,73124.2512,2.313239e+06,4.005763e+06,0.1697,357280.8529,5.0252,32.5985,4.9105,...,5.8489,5.9292,2.7585,1.096003e+06,1.427530e+06,1.692523e+06,4.0381,35.6369,45.7534,84.3433
BAW SJ Equity,0.8784,7.705000e+03,4142.0000,3.130300e+04,4.812200e+04,0.1198,6624.0000,4.1374,14.7108,4.1374,...,10.6581,9.6124,14.3336,4.502800e+04,1.184700e+04,1.681900e+04,1.7885,24.6187,41.3277,70.4382
BHG SJ Equity,0.5479,2.852730e+05,134871.0498,9.921380e+05,1.904628e+06,0.8369,486944.0718,32.0082,50.9114,24.3114,...,21.8961,20.3470,-17.3293,9.564535e+05,4.201441e+05,9.124902e+05,0.8155,22.0591,31.5273,46.0437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UGPA3 BS Equity,3.3751,1.098727e+04,2304.6800,2.422215e+04,3.825197e+04,0.2896,5720.7110,3.5200,4.5385,3.5200,...,13.8756,12.2053,-12.2436,1.260487e+05,1.329195e+04,1.402983e+04,2.3235,34.7484,48.6497,94.7407
USIM5 BS Equity,0.6896,5.855413e+03,167.9600,1.361231e+04,4.016175e+04,0.7852,1592.4070,2.3043,5.7616,2.3043,...,6.4728,6.5389,-14.8817,2.763835e+04,6.023373e+03,2.654944e+04,3.7826,14.9978,18.4920,22.6874
VALE3 BS Equity,0.4576,6.246400e+04,4940.0000,2.576590e+05,4.559840e+05,0.9927,85917.0000,14.8800,41.2931,14.6366,...,16.9916,23.7007,-8.1419,2.080660e+05,6.740400e+04,1.983250e+05,0.7845,14.7821,25.3657,33.9866
VAMO3 BS Equity,0.3247,1.083538e+04,881.6250,1.607353e+04,2.080883e+04,-0.2292,2668.1330,1.7422,43.8442,1.7422,...,11.6270,11.4471,23.8534,6.085482e+03,1.171701e+04,4.735295e+03,4.3915,56.3079,71.2180,247.4399


In [99]:
ft = pd.read_excel("dataaa.xlsx", index_col=0, parse_dates=True)

C:\Users\Thabang Ndhlovu\AppData\Local\Temp\ipykernel_772\536687646.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ft = pd.read_excel("dataaa.xlsx", index_col=0, parse_dates=True)


In [100]:
import joblib
import pandas as pd

study = {}
studies = {}

for company in features.index:
    ratios = features.loc[company][model_metrics].to_dict()    
    model = CreditRatingCalculator(model_inputs)
    model.calculate_credit_rating(ratios)
    
    # if company in tickers:
    #     credit_score = tree_model.predict(features.loc[company, model_metrics].to_frame().T).round(1)[0]
    #     credit_rating = determine_credit_rating(credit_score)
    # else:
    credit_score = model.credit_score
    credit_rating = model.credit_rating
    
    study[company] = {"model_class_scores": model.scores, "model_credit_scores": credit_score, "model_credit_ratings": credit_rating}
    studies[company] = {"model_inputs": ratios}
studies = pd.DataFrame(studies).T


In [101]:
study_df = studies.copy()

In [62]:
ba = study_df.iloc[:99]

In [45]:
b = pd.concat([ft, ba])

In [57]:
b = pd.read_excel("results.xlsx",index_col=0)

In [64]:
b = b.join(ba)

In [67]:
b.columns = b.columns.str.lower()

In [69]:
b.to_excel("results2.xlsx")

In [70]:
b

,security_name,gics_sector_name,model_credit_ratings,model_credit_scores,model_class_scores,rsk_bb_issuer_default,bb_5y_default_prob,rtg_moody_long_term,rtg_moody_long_term_date,rtg_sp_lt_lc_issuer_credit,rtg_sp_lt_lc_iss_cred_rtg_dt,country_full_name,book_val_per_sh,model_inputs
BLU SJ Equity,Blue Label Telecoms Ltd,Communication Services,Baa,5.365,"{'profitability': 9.0, 'leverage_coverage': 4....",HY1,0.074174,NaN,NaN,NaN,NaN,SOUTH AFRICA,5.296074,NaN
MCG SJ Equity,MultiChoice Group,Communication Services,Ba,5.880,"{'profitability': 6.0, 'leverage_coverage': 6....",IG7,0.024876,NaN,NaN,NaN,NaN,SOUTH AFRICA,13.086630,NaN
MTN SJ Equity,MTN Group Ltd,Communication Services,Baa,4.960,"{'profitability': 5.0, 'leverage_coverage': 5....",IG7,0.020231,Ba2,2022/04/06,BB-,2020/05/12,SOUTH AFRICA,77.065739,NaN
TKG SJ Equity,Telkom SA SOC Ltd,Communication Services,Ba,5.890,"{'profitability': 7.0, 'leverage_coverage': 5....",HY2,0.066731,Ba2,2023/12/19,BB,2020/05/12,SOUTH AFRICA,51.835067,NaN
VOD SJ Equity,Vodacom Group Ltd,Communication Services,A,4.345,"{'profitability': 4.0, 'leverage_coverage': 4....",IG2,0.007515,NaN,NaN,NaN,NaN,SOUTH AFRICA,47.916122,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VBBR3 BS Equity,Vibra Energia SA,Consumer Discretionary,B,6.720,"{'profitability': 9.0, 'leverage_coverage': 6....",IG5,0.013553,Ba1,2021/05/18,NaN,NaN,BRAZIL,14.109023,"{'oper_margin': 4.371973709242882, 'tot_debt_t..."
VIVA3 BS Equity,Vivara Participacoes SA,Consumer Discretionary,Baa,4.685,"{'profitability': 5.0, 'leverage_coverage': 4....",IG4,0.015635,NaN,NaN,NaN,NaN,BRAZIL,8.221579,"{'oper_margin': 20.605913188911963, 'tot_debt_..."
VIVT3 BS Equity,Telefonica Brasil SA,Communication Services,A,4.015,"{'profitability': 6.0, 'leverage_coverage': 2....",IG1,0.002467,NaN,NaN,NaN,NaN,BRAZIL,42.095769,"{'oper_margin': 15.217873360866074, 'tot_debt_..."
WEGE3 BS Equity,WEG SA,Industrials,Aa,3.405,"{'profitability': 6.0, 'leverage_coverage': 2....",IG1,0.000725,NaN,NaN,NaN,NaN,BRAZIL,4.133758,"{'oper_margin': 19.88091104121048, 'tot_debt_t..."


In [129]:
df =  pd.read_excel("results2 - Copy.xlsx", index_col=0)
ft = pd.read_excel("dataset/ibx_jalsh_features.xlsx", index_col=0)

In [123]:
a = df[["rtg_moody_long_term"]].dropna().index.to_list()
b = df[["rtg_sp_lt_lc_issuer_credit"]].dropna().index.to_list()

stocks = list(np.unique([a + b]))

In [126]:
study =df.loc[stocks]

In [138]:
n = ft.loc[ft.index[ft.index.isin(stocks)]]

In [143]:
d = n.dropna()

In [145]:
val = d.index.to_list()

In [148]:
dd = df.loc[val]

In [150]:
dd.sort_values(["country_full_name"])

,security_name,gics_sector_name,model_credit_ratings,model_credit_scores,model_class_scores,rsk_bb_issuer_default,bb_5y_default_prob,rtg_moody_long_term,rtg_moody_long_term_date,rtg_sp_lt_lc_issuer_credit,rtg_sp_lt_lc_iss_cred_rtg_dt,country_full_name,book_val_per_sh
MTN SJ Equity,MTN Group Ltd,Communication Services,Baa,4.960,"{'profitability': 5.0, 'leverage_coverage': 5....",IG7,0.020231,Ba2,2022/04/06,BB-,2020/05/12,SOUTH AFRICA,77.065739
TKG SJ Equity,Telkom SA SOC Ltd,Communication Services,Ba,5.890,"{'profitability': 7.0, 'leverage_coverage': 5....",HY2,0.066731,Ba2,2023/12/19,BB,2020/05/12,SOUTH AFRICA,51.835067
CYRE3 BS Equity,Cyrela Brazil Realty SA Empree,Consumer Discretionary,B,6.955,"{'profitability': 7.0, 'leverage_coverage': 7....",IG8,0.047074,NaN,NaN,NR,2022/07/29,BRAZIL,20.511897
VBBR3 BS Equity,Vibra Energia SA,Consumer Discretionary,B,6.720,"{'profitability': 9.0, 'leverage_coverage': 6....",IG5,0.013553,Ba1,2021/05/18,NaN,NaN,BRAZIL,14.109023
PPH SJ Equity,Pepkor Holdings Ltd,Consumer Discretionary,Ba,5.915,"{'profitability': 8.0, 'leverage_coverage': 5....",IG4,0.012459,Ba2,2022/04/14,NaN,NaN,SOUTH AFRICA,16.046087
CFR SJ Equity,Cie Financiere Richemont SA,Consumer Discretionary,A,4.270,"{'profitability': 6.0, 'leverage_coverage': 3....",IG3,0.011695,NaN,NaN,A+,2018/03/05,SWITZERLAND,NaN
ANH SJ Equity,Anheuser-Busch InBev SA/NV,Consumer Staples,A,4.275,"{'profitability': 4.0, 'leverage_coverage': 4....",IG2,0.010297,A3,2023/03/31,A-,2023/04/21,BELGIUM,NaN
ABEV3 BS Equity,Ambev SA,Consumer Staples,A,3.695,"{'profitability': 5.0, 'leverage_coverage': 2....",IG1,0.000295,Baa3,2023/11/13,BBB+,2023/12/20,BRAZIL,5.014099
BEEF3 BS Equity,Minerva SA/Brazil,Consumer Staples,B,7.470,"{'profitability': 8.0, 'leverage_coverage': 8....",HY5,0.113837,NaN,NaN,BB,2020/10/15,BRAZIL,0.297169
BRFS3 BS Equity,BRF SA,Consumer Staples,Caa,8.445,"{'profitability': 9.0, 'leverage_coverage': 8....",IG9,0.037099,Ba3,2024/03/18,BB,2023/12/20,BRAZIL,8.890105


In [162]:
df = dd.reset_index()

In [163]:
df['Sort_Key'] = df['index'].apply(lambda x: x.split()[1])

In [168]:
df_sorted = df.sort_values(by=['Sort_Key'], ascending=False) 


In [171]:
df_sorted = df_sorted.drop(["Sort_Key"], axis=1)

In [173]:
df = df_sorted.set_index("index")

In [176]:
import json

In [177]:
with open("hello_credit_config.json", "r") as f:
    file = json.load(f)

In [180]:
mappings = file['credit_rating_mappings']

In [191]:
import json

with open("hello_credit_config.json", "r") as f:
    file = json.load(f)
    
mappings = file['credit_rating_mappings']

# Invert the dictionary
invert_dict = {v: k for k, vals in mappings.items() for v in vals}

df['rtg_moody_long_term'] = df['rtg_moody_long_term'].map(invert_dict)
df['rtg_sp_lt_lc_issuer_credit'] = df['rtg_sp_lt_lc_issuer_credit'].map(invert_dict)
df['rsk_bb_issuer_default'] = df['rsk_bb_issuer_default'].map(invert_dict)

In [185]:
df['rtg_moody_long_term'] = df['rtg_moody_long_term'].map(invert_dict)
df['rtg_sp_lt_lc_issuer_credit'] = df['rtg_sp_lt_lc_issuer_credit'].map(invert_dict)
df['rsk_bb_issuer_default'] = df['rsk_bb_issuer_default'].map(invert_dict)

In [187]:
df['rtg_sp_lt_lc_issuer_credit'] = df['rtg_sp_lt_lc_issuer_credit'].map(invert_dict)
df['rsk_bb_issuer_default'] = df['rsk_bb_issuer_default'].map(invert_dict)

In [193]:
df['rsk_bb_issuer_default'] = df['rsk_bb_issuer_default'].map(invert_dict)

In [198]:
c = file['credit_rating_scale']

In [ ]:
df['']

In [199]:
df['rtg_moody_long_term'] = df['rtg_moody_long_term'].map(c)

In [206]:
df['rsk_bb_issuer_default']= df['rsk_bb_issuer_default'].map(c)

In [210]:
df['model_credit_ratings'] = df['model_credit_ratings'].map(c)

In [212]:
df.to_excel("demo2.xlsx")

In [219]:
df = df.sort_values(["gics_sector_name", "country_full_name", "model_credit_ratings","rtg_moody_long_term", "rtg_sp_lt_lc_issuer_credit"])

In [220]:
df.to_excel("demo3.xlsx")

In [215]:
df

,security_name,gics_sector_name,model_credit_ratings,model_credit_scores,model_class_scores,rsk_bb_issuer_default,bb_5y_default_prob,rtg_moody_long_term,rtg_moody_long_term_date,rtg_sp_lt_lc_issuer_credit,rtg_sp_lt_lc_iss_cred_rtg_dt,country_full_name,book_val_per_sh
index,,,,,,,,,,,,,
TKG SJ Equity,Telkom SA SOC Ltd,Communication Services,6.5,5.890,"{'profitability': 7.0, 'leverage_coverage': 5....",7.5,0.066731,6.5,2023/12/19,6.5,2020/05/12,SOUTH AFRICA,51.835067
GLN SJ Equity,Glencore PLC,Materials,5.5,4.680,"{'profitability': 7.0, 'leverage_coverage': 3....",4.5,0.019592,5.5,2023/11/20,5.5,2018/05/29,SWITZERLAND,NaN
AGL SJ Equity,Anglo American PLC,Materials,5.5,4.575,"{'profitability': 6.0, 'leverage_coverage': 5....",4.5,0.032993,5.5,2023/12/14,5.5,2024/03/08,BRITAIN,20.668427
ANG SJ Equity,Anglogold Ashanti Plc,Materials,5.5,4.674,"{'profitability': 8.0, 'leverage_coverage': 5....",4.5,0.021778,5.5,2023/09/26,NaN,NaN,BRITAIN,8.865255
ANH SJ Equity,Anheuser-Busch InBev SA/NV,Consumer Staples,4.5,4.275,"{'profitability': 4.0, 'leverage_coverage': 4....",4.5,0.010297,4.5,2023/03/31,4.5,2023/04/21,BELGIUM,NaN
BHG SJ Equity,BHP Group Ltd,Materials,4.5,4.275,"{'profitability': 4.0, 'leverage_coverage': 4....",4.5,0.004565,4.5,2023/02/28,4.5,2022/06/01,AUSTRALIA,NaN
BVT SJ Equity,Bidvest Group Ltd/The,Industrials,6.5,6.055,"{'profitability': 8.0, 'leverage_coverage': 6....",4.5,0.011509,6.5,2022/04/06,NaN,NaN,SOUTH AFRICA,100.080076
CFR SJ Equity,Cie Financiere Richemont SA,Consumer Discretionary,4.5,4.270,"{'profitability': 6.0, 'leverage_coverage': 3....",4.5,0.011695,NaN,NaN,4.5,2018/03/05,SWITZERLAND,NaN
GFI SJ Equity,Gold Fields Ltd,Materials,4.5,4.820,"{'profitability': 4.0, 'leverage_coverage': 3....",4.5,0.021342,5.5,2022/04/06,5.5,2022/04/28,SOUTH AFRICA,5.009396
